## 載入模型

In [678]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [679]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [680]:
# 載入模型與標準化器
base_model = load_model('DNN_best_model4mcAP_2week_to_3week_1dataPerRP.h5')


scaler = joblib.load('Model_SAVE/4 mc AP RSSI FTM StdDev 2024-12-14 data train/scaler_4mcAP.pkl')

c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [681]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_44 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 49)             │         3,185 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,595 (80.45 KB)

 Trainable params: 3,185 (12.44 KB)

 Non-trainable params: 17,408 (68.00 KB)

 Optimizer params: 2 (12.00 B)

In [682]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [683]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = [
        'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
        'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [684]:
# 讀取測試資料
test_file_path = "timestamp_allignment_Balanced_2025_01_10_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2025_01_10"
modelname = "DNN 4 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,3999.0,7339.0,14219.0,-303.0,-65.0,-68.0,-65.0,-50.0,986.0,295.0,580.0,968.0
1,1-1,3921.0,7368.0,12835.0,-420.0,-65.0,-68.0,-64.0,-54.0,1071.0,1013.0,765.0,671.0
2,1-1,4038.0,6928.0,12510.0,-651.0,-65.0,-69.0,-64.0,-52.0,986.0,375.0,562.0,770.0
3,1-1,4038.0,7163.0,13955.0,-381.0,-65.0,-68.0,-65.0,-54.0,1042.0,267.0,527.0,555.0
4,1-1,4077.0,7163.0,13281.0,-342.0,-65.0,-68.0,-64.0,-54.0,996.0,137.0,826.0,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20673,9-11,1694.0,5083.0,2666.0,5438.0,-55.0,-67.0,-50.0,-66.0,294.0,1093.0,369.0,2384.0
20674,9-11,NaN,5229.0,NaN,NaN,NaN,-65.0,NaN,NaN,NaN,1125.0,NaN,NaN
20675,9-11,2007.0,5346.0,2627.0,4696.0,-55.0,-66.0,-50.0,-66.0,152.0,1073.0,482.0,463.0
20676,9-11,1929.0,5434.0,2549.0,4735.0,-55.0,-65.0,-51.0,-66.0,216.0,964.0,568.0,481.0


In [685]:
# 資料前處理 (一): 刪除前後n筆資料
n = 10
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

C:\Users\吳定洋\AppData\Local\Temp\ipykernel_13772\195304898.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)


Number of rows with NaN values: 974
Number of rows with NaN values: 0


,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,4038.0,7358.0,13018.0,-1855.0,-65.0,-68.0,-64.0,-50.0,965.0,877.0,314.0,2003.0
1,1-1,1,3960.0,7241.0,13252.0,-342.0,-65.0,-68.0,-66.0,-54.0,1056.0,880.0,160.0,483.0
2,1-1,2,3843.0,7202.0,13838.0,-303.0,-65.0,-68.0,-64.0,-54.0,1053.0,876.0,714.0,583.0
3,1-1,3,3960.0,7397.0,14307.0,-342.0,-65.0,-68.0,-64.0,-54.0,938.0,105.0,265.0,522.0
4,1-1,4,3999.0,7075.0,13174.0,-460.0,-65.0,-68.0,-66.0,-54.0,959.0,303.0,341.0,637.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19693,9-11,19693,1850.0,5464.0,2666.0,4852.0,-55.0,-65.0,-50.0,-67.0,215.0,959.0,409.0,497.0
19694,9-11,19694,1967.0,5112.0,2353.0,4881.0,-55.0,-66.0,-50.0,-67.0,141.0,1158.0,709.0,577.0
19695,9-11,19695,1968.0,5024.0,2366.0,4705.0,-55.0,-67.0,-48.0,-66.0,175.0,1166.0,671.0,491.0
19696,9-11,19696,1772.0,5434.0,2666.0,5008.0,-55.0,-66.0,-52.0,-67.0,151.0,861.0,519.0,483.0


In [686]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [687]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

y_test_numeric

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


0        10
1        10
2        10
3        10
4        10
         ..
19693    32
19694    32
19695    32
19696    32
19697    32
Name: Label, Length: 19698, dtype: int64

In [688]:
# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])

# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [689]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

## 重新訓練model

### train validation 切

In [690]:
# 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
from sklearn.model_selection import StratifiedShuffleSplit

# 設定每個類別的固定樣本數
N_train = 1 # 訓練集每個類別至少要有 N_train 筆資料
N_val = 0
test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_test_scaled)
data['label'] = y_test_numeric  # 加入 label 欄位

print((data['label'] == 10).sum())  # 直接計算 True 的數量

402


In [691]:
# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_test_scaled)
data['label'] = y_test_numeric  # 加入 label 欄位

# data

In [692]:
# 儲存訓練集（但這時包含 validation 的部分）
# train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
# 儲存訓練集（確保每個類別只選 1 筆，若某類少於 N_train，則取全部）
train_data_full = data.groupby('label', group_keys=False).apply(lambda x: x.sample(n=min(N_train, len(x)), replace=False, random_state=42)).reset_index(drop=True)
# 確保 `train_data_full` 內部的 `label` 數量正確
print(train_data_full['label'].value_counts())  # 每個類別應該剛好 1 筆
# train_data_full

label
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
Name: count, dtype: int64


C:\Users\吳定洋\AppData\Local\Temp\ipykernel_13772\1501761780.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data_full = data.groupby('label', group_keys=False).apply(lambda x: x.sample(n=min(N_train, len(x)), replace=False, random_state=42)).reset_index(drop=True)


In [693]:
# 剩下的資料（未被抽入 train_data_full 的部分）直接作為 test set
remaining_data = data.drop(train_data_full.index)
print(len(remaining_data))

19649


In [694]:
# 若 N_train 太小，則 validation set 直接從 test set 取一部分
if N_val > 0:
    sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / len(remaining_data), random_state=42)
    val_index, test_index = next(sss.split(remaining_data.drop(columns=['label']), remaining_data['label']))
    val_data = remaining_data.iloc[val_index]
    remaining_data = remaining_data.iloc[test_index]
else:
    val_data = pd.DataFrame(columns=data.columns)  # 若沒有 validation data，建立空 DataFrame


In [695]:
# 提取 features 和 labels
X_train, y_train = train_data_full.drop(columns=['label']).values, train_data_full['label'].values
X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values

# 確認切分後的數據數量
print(f"Training set: {len(X_train)} samples, {len(np.unique(y_train))} unique labels")
print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

Training set: 49 samples, 49 unique labels
Validation set: 0 samples, 0 unique labels
Test set: 19649 samples, 49 unique labels


In [696]:
# 計算每個 Set 內各 Label 的資料數量
train_label_counts = pd.Series(y_train).value_counts().sort_index()
val_label_counts = pd.Series(y_val).value_counts().sort_index()
test_label_counts = pd.Series(y_test).value_counts().sort_index()

# 確保所有 Labels 都有出現在三個 Set 裡
all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
label_distribution = pd.DataFrame(index=all_labels)

label_distribution["Training Set"] = train_label_counts
label_distribution["Validation Set"] = val_label_counts
label_distribution["Test Set"] = test_label_counts

print(test_label_counts)


0     402
1     402
2     402
3     402
4     402
5     402
6     402
7     402
8     402
9     402
10    353
11    402
12    402
13    402
14    402
15    402
16    402
17    402
18    402
19    402
20    402
21    402
22    402
23    402
24    402
25    402
26    402
27    402
28    402
29    402
30    402
31    402
32    402
33    402
34    402
35    402
36    402
37    402
38    402
39    402
40    402
41    402
42    402
43    402
44    402
45    402
46    402
47    402
48    402
Name: count, dtype: int64


In [697]:
# 用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）
label_distribution = label_distribution.fillna(0).astype(int)

from IPython.display import display
display(label_distribution)

,Training Set,Validation Set,Test Set
0,1,0,402
1,1,0,402
2,1,0,402
3,1,0,402
4,1,0,402
5,1,0,402
6,1,0,402
7,1,0,402
8,1,0,402
9,1,0,402


## 訓練

In [698]:
import time
# 記錄開始時間
start_time = time.time()

In [699]:
from tensorflow.keras.callbacks import EarlyStopping

if N_val > 0:
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
else:
    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)


In [700]:
# 確保變數命名一致
X_train_small = X_train  # 確保這裡用的變數和前面一致
y_train_small = y_train

In [ ]:
# 設定 batch_size
batch_size = min(32, max(8, len(X_train_small) // 2))  # 避免 batch size 過大

In [ ]:
if N_val > 0:
    base_model.fit(X_train_small, y_train_small,validation_data=(X_val, y_val), epochs=10000, batch_size=batch_size, callbacks=[early_stop])
else:
    base_model.fit(X_train_small, y_train_small, epochs=10000, batch_size=batch_size, callbacks=[early_stop])

Epoch 1/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4796 - loss: 8.7288 
Epoch 2/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4932 - loss: 7.6997
Epoch 3/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5381 - loss: 7.6127
Epoch 4/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5485 - loss: 7.0733
Epoch 5/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5068 - loss: 7.2430
Epoch 6/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5381 - loss: 6.6573
Epoch 7/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5276 - loss: 6.6620
Epoch 8/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5172 - loss: 7.0091
Epoch 9/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4756 - loss: 7.7524
Epoch 10/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4964 - loss: 6.8503
Epoch 11/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4964 - loss: 6.7247
Epoch 12/10000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/ste

In [ ]:
# 記錄結束時間
end_time = time.time()

# 計算訓練時間（秒）
training_time = end_time - start_time
print(f"訓練時間：{training_time:.2f} 秒")

訓練時間：317.78 秒


In [ ]:
print(len(X_test))

19453


In [ ]:
loss, accuracy = base_model.evaluate(X_test, y_test)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

219/608 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step - accuracy: 0.9358 - loss: 0.4756

608/608 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.8814 - loss: 0.9584
Evaluation on 90% unused data - Loss: 1.6902, Accuracy: 0.8074


In [ ]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [ ]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

610/610 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step


['1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-2',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-4',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',


In [ ]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,4385.000000,7075.00000,12054.000000,-342.0,-53.000000,-71.000000,-60.000000,-47.0,2406.000000,1989.000000,1566.000000,1455.0
1,1-1,1,3999.000000,7192.00000,10518.000000,-342.0,-61.000000,-71.000000,-64.000000,-47.0,1083.000000,2197.000000,1001.000000,1488.0
2,1-1,2,4116.000000,9155.00000,10440.000000,-303.0,-60.000000,-71.000000,-64.000000,-45.0,1073.000000,1509.000000,1253.000000,1468.0
3,1-1,3,3999.000000,6811.00000,10478.000000,-342.0,-61.000000,-71.000000,-65.000000,-47.0,1161.000000,2386.000000,1177.000000,1531.0
4,1-1,4,4038.000000,6841.00000,10479.000000,-303.0,-61.000000,-71.000000,-64.000000,-47.0,1029.000000,2075.000000,1131.000000,1509.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19497,9-11,19497,2202.000000,8218.00000,2588.000000,4617.0,-56.000000,-66.000000,-51.000000,-69.0,88.000000,384.000000,488.000000,292.0
19498,9-11,19498,2202.000000,8139.00000,2666.000000,4539.0,-56.000000,-66.000000,-51.000000,-69.0,88.000000,421.000000,419.000000,330.0
19499,9-11,19499,2085.000000,8335.00000,2588.000000,5057.0,-56.000000,-66.000000,-51.000000,-68.0,142.000000,356.000000,442.000000,1292.0
19500,9-11,19500,2081.512821,8234.04359,2576.604113,4735.0,-55.335897,-65.969231,-50.969152,-68.0,190.787179,488.812821,476.794344,217.0


In [ ]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 4 mcAPs BEST_2025_01_03.json

Test Data Mean MDE: 0.2963 meters


In [ ]:
base_model.save(f"DNN_best_model4mcAP_3week_to_4week_1dataPerRP.h5")

### test validation 切

In [ ]:
# # 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
# from sklearn.model_selection import StratifiedShuffleSplit

# # 設定每個類別的固定樣本數
# N_train = 5 # 訓練集每個類別至少要有 N_train 筆資料
# N_val = 2
# test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

# # 轉為 DataFrame 方便操作
# data = pd.DataFrame(X_test_scaled)
# data['label'] = y_test_numeric  # 加入 label 欄位

# print((data['label'] == 10).sum())  # 直接計算 True 的數量


In [ ]:
# data

In [ ]:
# # 儲存訓練集
# train_data = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
# # print(train_data.index)  # 查看 train_data 的索引

# print((train_data['label'] == 10).sum())  # 計算 label 為 10 的數量


In [ ]:
# remaining_data = data.drop(train_data.index)
# print(remaining_data['label'].value_counts().get(10, 0))  # 應該是 348


In [ ]:
# # 取得剩餘資料的 features 和 labels
# X_remaining = remaining_data.drop(columns=['label'])
# y_remaining = remaining_data['label']

# print(y_remaining.value_counts().get(10, 0))

In [ ]:
# # 使用 StratifiedShuffleSplit 來確保驗證集與測試集的類別比例一致
# sss = StratifiedShuffleSplit(n_splits=1, test_size=test_val_ratio, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val/N_train, random_state=42)
# for val_index, test_index in sss.split(X_remaining, y_remaining):
#     X_val, X_test = X_remaining.iloc[val_index], X_remaining.iloc[test_index]
#     y_val, y_test = y_remaining.iloc[val_index], y_remaining.iloc[test_index]

# # 轉換回 NumPy 陣列
# X_train_small, y_train_small = train_data.drop(columns=['label']).values, train_data['label'].values
# X_val, y_val = X_val.values, y_val.values
# X_test, y_test = X_test.values, y_test.values

# # 確認切分後的數據數量
# print(f"Training set: {len(X_train_small)} samples, {len(np.unique(y_train_small))} unique labels")
# print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
# print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

In [ ]:
# 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
X_train_small, X_unused, y_train_small, y_unused = train_test_split(
    X_test_scaled, y_test_numeric, test_size=0.99, random_state=42, stratify=y_test_numeric
)
# 再切出一半作為validation set and test set
X_val, X_test, y_val, y_test = train_test_split(
    X_unused, y_unused, test_size=0.5, random_state=42, stratify=y_unused
)

In [ ]:
# # 檢查各組資料比例
# print(f"Training set: {len(X_train_small)} samples")
# print(f"Validation set: {len(X_val)} samples")
# print(f"Test set: {len(X_test)} samples")

In [ ]:
# import pandas as pd

# # 計算每個 Set 內各 Label 的資料數量
# train_label_counts = pd.Series(y_train_small).value_counts().sort_index()
# val_label_counts = pd.Series(y_val).value_counts().sort_index()
# test_label_counts = pd.Series(y_test).value_counts().sort_index()

# # # 印出統計結果
# # print("Training Set - Label Counts:")
# # print(train_label_counts)

# # print("\nValidation Set - Label Counts:")
# # print(val_label_counts)

# # print("\nTest Set - Label Counts:")
# # print(test_label_counts)

# # 確保所有 Labels 都有出現在三個 Set 裡
# all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
# label_distribution = pd.DataFrame(index=all_labels)

# label_distribution["Training Set"] = train_label_counts
# label_distribution["Validation Set"] = val_label_counts
# label_distribution["Test Set"] = test_label_counts

# # 用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）
# label_distribution = label_distribution.fillna(0).astype(int)

# from IPython.display import display
# display(label_distribution)
